In [4]:
# Cell to install necessary packages
! pip install wikipedia-api requests beautifulsoup4
%pip install python-dotenv
%pip install --quiet langchain openai langchain-openai unstructured
%pip install --upgrade openai



You should consider upgrading via the '/Users/ATIFHANIF/.pyenv/versions/3.7.12/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/ATIFHANIF/.pyenv/versions/3.7.12/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement langchain-openai (from versions: none)
ERROR: No matching distribution found for langchain-openai
You should consider upgrading via the '/Users/ATIFHANIF/.pyenv/versions/3.7.12/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
  Using cached openai-1.33.0-py3-none-any.whl (325 kB)
  Using cached pydantic-2.5.3-py3-none-any.whl (381 kB)
  Using cached cached_property-1.5.2-py2.py3-none-any.whl (7.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl (20 kB)
  Using cached tqdm-4.66.4-py3-none-any.whl (78 kB)
  Using cached httpx-0.24.1-py3-no

In [5]:
import wikipediaapi
import requests
from bs4 import BeautifulSoup
import os
from datetime import datetime
from dotenv import load_dotenv

In [13]:
! pip install --upgrade pip

! pip install openai

! pip install nltk

! pip install wikipedia-api requests beautifulsoup4
! pip install python-dotenv
! pip install --quiet langchain openai langchain-openai unstructured
! pip install --upgrade openai




ERROR: Ignored the following versions that require a different python version: 0.0.1 Requires-Python >=3.8.1,<4.0; 0.0.100 Requires-Python >=3.8.1,<4.0; 0.0.101 Requires-Python >=3.8.1,<4.0; 0.0.101rc0 Requires-Python >=3.8.1,<4.0; 0.0.102 Requires-Python >=3.8.1,<4.0; 0.0.102rc0 Requires-Python >=3.8.1,<4.0; 0.0.103 Requires-Python >=3.8.1,<4.0; 0.0.104 Requires-Python >=3.8.1,<4.0; 0.0.105 Requires-Python >=3.8.1,<4.0; 0.0.106 Requires-Python >=3.8.1,<4.0; 0.0.107 Requires-Python >=3.8.1,<4.0; 0.0.108 Requires-Python >=3.8.1,<4.0; 0.0.109 Requires-Python >=3.8.1,<4.0; 0.0.110 Requires-Python >=3.8.1,<4.0; 0.0.111 Requires-Python >=3.8.1,<4.0; 0.0.112 Requires-Python >=3.8.1,<4.0; 0.0.113 Requires-Python >=3.8.1,<4.0; 0.0.114 Requires-Python >=3.8.1,<4.0; 0.0.115 Requires-Python >=3.8.1,<4.0; 0.0.116 Requires-Python >=3.8.1,<4.0; 0.0.117 Requires-Python >=3.8.1,<4.0; 0.0.118 Requires-Python >=3.8.1,<4.0; 0.0.119 Requires-Python >=3.8.1,<4.0; 0.0.120 Requires-Python >=3.8.1,<4.0; 0.0.1

In [12]:
# Cell to import necessary libraries
import re
import os
import openai
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
nltk.download('stopwords')
from langchain.chat_models import ChatOpenAI
from langchain.chains.openai_functions import (
    create_structured_output_runnable,
    create_structured_output_chain,
)




ModuleNotFoundError: No module named 'langchain_openai'

In [ ]:
#################### Updated tool for Scrapping ############################

def fetch_page(url):
    response = requests.get(url)
    return response.text

def parse_page(html, base_url):
    soup = BeautifulSoup(html, 'html.parser')
    
    # Extract the title for the directory name
    title_tag = soup.find('h1')
    title_text = title_tag.text.replace('/', '_')  # Replace '/' in titles with '_' to avoid path issues
    
    # Extract all text from the main content
    content = soup.find('div', {'id': 'mw-content-text'})
    text = content.get_text() if content else ""
    
    # Find links within the content that include the main title
    links = [a['href'] for a in soup.find_all('a', href=True) if a.text and title_text in a.text]
    
    # Convert relative links to absolute
    links = ['https://en.wikipedia.org' + link if link.startswith('/wiki/') else link for link in links]
    
    return text, links, title_text
    

def scrape_wikipedia(start_url):
    html = fetch_page(start_url)
    text, links, title = parse_page(html, start_url)
    
    # For simplicity, just fetch text from the first few relevant links
    for link in links[:10]:  # Limit to first 10 links to avoid too many requests
        html = fetch_page(link)
        page_text, _ , _ = parse_page(html, link)  # We don't follow further links here
        text += "\n\n" + page_text

    base_dir = os.path.join('data/scraped_data', title)
    os.makedirs(base_dir, exist_ok=True)  # Create directory if it does not exist
    
    # Format current datetime for the filename
    now = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    filename = f"{title}_{now}_Wikipedia.txt"
    filepath = os.path.join(base_dir, filename)
    
    # Save the content to a text file
    with open(filepath, 'w', encoding='utf-8') as file:
        file.write(text)

    return text, title


In [ ]:

# Example usage
url = 'https://en.wikipedia.org/wiki/Scent_of_a_Woman_(1992_film)'
collected_text, title = scrape_wikipedia(url)
print(len(collected_text))
print(f"Data collected for {title} and saved to the respective directory.")


In [ ]:
##################### For cleaning scrapped Data and dividing it into chunks on the basis of tokenization ############################
import os
import re
from nltk.tokenize import word_tokenize

def normalize_text(text):
    # Normalize whitespace and remove special characters
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.lower()

def chunk_text(text, chunk_size=8000):
    words = word_tokenize(text)
    chunks = [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    # Handle the last chunk; merge with previous if too small
    if len(chunks) > 1 and len(word_tokenize(chunks[-1])) < 1000:
        chunks[-2] += ' ' + chunks.pop()
    return chunks

def save_chunks(chunks, base_dir, filename):
    os.makedirs(base_dir, exist_ok=True)
    for i, chunk in enumerate(chunks):
        chunk_filename = f"{filename}_chunk{i+1}.txt"
        chunk_path = os.path.join(base_dir, chunk_filename)
        with open(chunk_path, 'w', encoding='utf-8') as file:
            file.write(chunk)

def clean_data(scraped_dir, cleaned_dir):
    for title in os.listdir(scraped_dir):
        title_path = os.path.join(scraped_dir, title)
        cleaned_title_path = os.path.join(cleaned_dir, title)
        
        if not os.path.exists(cleaned_title_path):
            os.makedirs(cleaned_title_path, exist_ok=True)
        
        for file in os.listdir(title_path):
            file_path = os.path.join(title_path, file)
            cleaned_file_path = os.path.join(cleaned_title_path, file.replace('.txt', ''))
            
            # Check if already cleaned
            if not any(f.startswith(file.replace('.txt', '')) for f in os.listdir(cleaned_title_path)):
                with open(file_path, 'r', encoding='utf-8') as f:
                    text = f.read()
                
                # Clean the text
                cleaned_text = normalize_text(text)
                chunks = chunk_text(cleaned_text)
                
                # Save cleaned chunks
                save_chunks(chunks, cleaned_title_path, file.replace('.txt', ''))
                print(f"Data cleaned and saved for {file_path}")
            else:
                print(f"Already cleaned data present for {file_path}")



In [ ]:

# # Example usage
# scraped_dir = 'data/scraped_data'
# cleaned_dir = 'data/cleaned_data'
# clean_data(scraped_dir, cleaned_dir)

In [ ]:
# Cell to define the function to remove unnecessary characters
def remove_unnecessary_characters(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text


In [ ]:
# Cell to define the main cleaning function
def clean_data(text):
    context = "This text is a part of a formal document and should be cleaned accordingly.You are a helpful assistant. Clean and standardize the following text to make it more readable and consistent."
    text = normalize_text(text)
    text = remove_unnecessary_characters(text)
    # text = context_based_cleaning(text,context)
    return text


In [ ]:
# Cell to create directories for cleaned data based on title and source
def create_cleaned_directory_structure(base_dir, title, source):
    dir_path = os.path.join(base_dir, title, source)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    return dir_path


In [ ]:
# Cell to clean the scraped data and save it in the cleaned_data directory
def clean_and_save_scraped_data(scraped_dir, cleaned_dir):
    for root, dirs, files in os.walk(scraped_dir):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = f.read()
                
                cleaned_data = clean_data(data)
                
                # Extract title and source from the file path
                relative_path = os.path.relpath(file_path, scraped_dir)
                title, source = relative_path.split(os.sep)[:2]
                
                cleaned_dir_path = create_cleaned_directory_structure(cleaned_dir, title, source)
                cleaned_file_path = os.path.join(cleaned_dir_path, file)
                
                with open(cleaned_file_path, 'w', encoding='utf-8') as f:
                    f.write(cleaned_data)
                
                print(f"Cleaned data saved to: {cleaned_file_path}")


In [ ]:

scraped_dir = "data/scraped_data"
cleaned_dir = "data/cleaned_data"

clean_and_save_scraped_data(scraped_dir, cleaned_dir)


In [ ]:
####################### For Chunked Data to extract Entitiy ############################

import os
import json
from langchain_openai import ChatOpenAI

# Initialize ChatOpenAI with the model and API key
llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)

def read_files_from_directory(directory_path: str):
    """
    Read text files from the specified directory.
    """
    files_content = {}
    for root, dirs, files in os.walk(directory_path):
        for filename in files:
            if filename.endswith(".txt"):
                file_path = os.path.join(root, filename)
                with open(file_path, 'r', encoding='utf-8') as file:
                    files_content[file_path] = file.read()
    print(f"Read {len(files_content)} files from directory {directory_path}")
    return files_content


def extract_information(llm, text: str, title: str) -> dict:
    prompt = {
        "role": "system",
        "content": f"""
        I am analyzing the TV show '{title}' and need to extract detailed information about characters to understand plot dynamics better. Please list all identifiable characters from the text below. For each character, provide:
        - A short fragment of text that captures their essence or a significant action.
        - Potential aliases.
        Fragments should not mention lists of character names from the TV show or movie. Provide the smallest fragment with contextual meaning.
        The response should be concise and structured in JSON format to facilitate relationship analysis in future iterations.

        An example of a valid response:
        [
            {{
                "character": "Jon Snow",
                "aliases": ["Lord Snow", "The White Wolf"],
                "fragment": "Jon Snow pledges his life to the Night's Watch and refuses to leave even when tempted."
            }},
            {{
                "character": "Daenerys Targaryen",
                "aliases": ["Dany", "Khaleesi"],
                "fragment": "Daenerys sets sail for Westeros with her armies and dragons, aiming to reclaim her family's throne."
            }}
        ]
        """
    }
    
    user_prompt = {
        "role": "user",
        "content": f"Below is part of that Wikipedia page on the TV show {title}.\n\n{text}"
    }

    combined_prompts = [prompt, user_prompt]  # This should be a list of dictionaries

    response = llm.invoke(combined_prompts)
    try:
        response_json = json.loads(response.content)
        return response_json
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print(f"Received malformed JSON response: {response.content}")
        return []  # Return an empty list to indicate failure


def save_json_to_file(data: dict, file_path: str):
    """
    Save the JSON data to a file.
    """
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, indent=4)

# def process_cleaned_data(directory_path: str, title: str = None, is_file: bool = False):
#     """
#     Process all cleaned data files and extract information.
#     """
#     isProcess =True
#     if is_file:
#         title = title or os.path.basename(directory_path).split('.')[0]
#         entity_json_path = f"data/Entity_json/{title}.json"
#         if os.path.exists(entity_json_path):
#             user_input = input(f"The title '{title}' is already in Entity_json. Would you like to process it again? (yes/no): ")
#             if user_input.lower() != 'yes':
#                 print(f"Skipping processing for {title}.")
#                 return
#         content = read_files_from_directory(directory_path)
#         extracted_info = extract_information(llm, content, title)
#         save_json_to_file(extracted_info, entity_json_path)
#         print(f"Processed and saved: {entity_json_path}")
#     else:
#         # Process a directory
#         files_content = read_files_from_directory(directory_path)
#         if not files_content:
#             print("No files found in the directory.")
#             return
#         for filepath, content in files_content.items():
#             title = os.path.basename(filepath).split('_')[0]  # Derive title from filename
#             print(title)
#             entity_json_path = f"data/Entity_json/{title}"
#             print(entity_json_path)
#             if os.path.exists(entity_json_path):
                
#                 user_input = input(f"The title '{title}' is already in Entity_json. Would you like to process it again? (yes/no): ")
#                 if user_input.lower() != 'yes':
#                     print(f"Skipping processing for {title}.")
#                     continue
                 
#             # extracted_info = extract_information(llm, content, title)
#             # save_json_to_file(extracted_info, entity_json_path)
#             print(f"Processed and saved: {entity_json_path}")



def process_cleaned_data(cleaned_data_dir: str):
    """
    Process all cleaned data directories and extract information.
    """
    for title in os.listdir(cleaned_data_dir):
        title_dir_path = os.path.join(cleaned_data_dir, title)
        entity_json_dir_path = os.path.join("data/Entity_json", title)
        
        if os.path.exists(entity_json_dir_path):
            user_input = input(f"The title '{title}' is already in Entity_json. Would you like to process it again? (yes/no): ")
            if user_input.lower() != 'yes':
                print(f"Skipping processing for all files under the title '{title}'.")
                continue
            else:
                # Remove existing JSON files to replace with new ones
                for file in os.listdir(entity_json_dir_path):
                    if file.endswith('.json'):
                        os.remove(os.path.join(entity_json_dir_path, file))

        files_content = read_files_from_directory(title_dir_path)
        for filepath, content in files_content.items():
            extracted_info = extract_information(llm, content, title)
            json_filename = os.path.basename(filepath).replace(".txt", ".json")
            json_filepath = os.path.join(entity_json_dir_path, json_filename)
            save_json_to_file(extracted_info, json_filepath)
            print(f"Processed and saved: {json_filepath}")


In [ ]:

# Example usage
# Process a directory
process_cleaned_data("data/cleaned_data")

In [ ]:
####################### For CHunked Data to extract Events ############################

def extract_narrative_elements(llm, text: str, title: str) -> dict:
    prompt = {
        "role": "system",
        "content": f"""
        I am analyzing the TV show '{title}' for the first time and need to extract detailed information about key narrative elements such as objects, locations, addresses, scenes, and events. Please identify these elements from the text below and provide:
        - A concise description or identification of the element.
        - A list of excerpts that discuss this element, ensuring each excerpt provides meaningful context.

        The response should be concise and structured in JSON format to facilitate analysis and visualization of these narrative elements in future iterations.

        An example of a valid response:
        [
            {{
                "element": "Shooting scene",
                "description": "Episode 3, Season 4",
                "excerpts": [
                    "The intense shooting scene in the third episode of the fourth season was pivotal to the plot development.",
                    "During the shootout, the main character's dilemma comes to a head, forcing a decision that changes the course of the story."
                ]
            }},
            {{
                "element": "Central Park",
                "description": "Location",
                "excerpts": [
                    "Several key discussions between the protagonists occur in Central Park, serving as a backdrop to their evolving relationships.",
                    "Central Park is depicted in multiple scenes as a place of reflection and confrontation among the characters."
                ]
            }}
        ]
        """
    }
    
    user_prompt = {
        "role": "user",
        "content": f"Below is part of that Wikipedia page on the TV show {title}.\n\n{text}"
    }

    combined_prompts = [prompt, user_prompt]  # This should be a list of dictionaries

    response = llm.invoke(combined_prompts)
    try:
        response_json = json.loads(response.content)
        return response_json
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print(f"Received malformed JSON response: {response.content}")
        return []  # Return an empty list to indicate failure




def process_cleaned_data_Events(directory_path: str, title: str = None, is_file: bool = False):
    """Process all cleaned data files and extract narrative information."""
    if is_file:
        with open(directory_path, 'r', encoding='utf-8') as file:
            content = file.read()
        title = title or os.path.basename(directory_path).split('.')[0]
        extracted_info = extract_narrative_elements(llm, content, title)
        json_filename = os.path.basename(directory_path).replace(".txt", ".json")
        json_filepath = os.path.join("data/Events_json", json_filename)
        save_json_to_file(extracted_info, json_filepath)
        print(f"Processed and saved: {json_filepath}")
    else:
        files_content = read_files_from_directory(directory_path)
        if not files_content:
            print("No files found in the directory.")
            return
        for filepath, content in files_content.items():
            title = os.path.basename(filepath).split('_')[0]  # Derive title from filename
            extracted_info = extract_narrative_elements(llm, content, title)
            json_filename = os.path.basename(filepath).replace(".txt", ".json")
            json_filepath = os.path.join("data/Events_json", title, json_filename)
            save_json_to_file(extracted_info, json_filepath)
            print(f"Processed and saved: {json_filepath}")



In [ ]:

process_cleaned_data_Events("data/cleaned_data")

In [ ]:
############################# Merge Entities ##############################

import os
import json
from langchain_openai import ChatOpenAI

# Initialize the language model

def merge_json_files(directory_path):
    """Merge all JSON files in a directory into a single JSON list."""
    merged_data = []
    if not os.path.exists(directory_path):
        print(f"No directory found at {directory_path}")
        return merged_data

    for file in os.listdir(directory_path):
        if file.endswith('.json'):
            with open(os.path.join(directory_path, file), 'r', encoding='utf-8') as f:
                try:
                    data = json.load(f)
                    if data:  # Ensure that data is not empty
                        merged_data.extend(data)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON from file {file}: {e}")
    return merged_data

def process_title_entities(title_directory):
    """Process all entity JSON files for a given title."""
    merged_data = merge_json_files(title_directory)
    if not merged_data:
        print("No data found to merge.")
        return

    title = os.path.basename(title_directory)
    # Save the merged data
    merged_dir = os.path.join('data/Merged_Entity_json', title)
    os.makedirs(merged_dir, exist_ok=True)
    merged_file_path = os.path.join(merged_dir, f"{title}_Merged.json")
    with open(merged_file_path, 'w', encoding='utf-8') as f:
        json.dump(merged_data, f, indent=4)
    print(f"Merged data saved to {merged_file_path}")




In [ ]:
# Example usage
title_directory = 'data/Entity_json/Interstellar (film)'  # Specify the subfolder for a specific title
process_title_entities(title_directory)


In [ ]:
############################### For Deduping without LLM ##############################
import json
from collections import defaultdict

def read_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def write_json(data, file_path):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)

def merge_entries(entries):
    # Dictionary to hold merged characters
    merged_characters = {}

    for entry in entries:
        # Identify character key
        character_key = entry['character'].lower()
        
        # If the character is already added, merge the data
        if character_key in merged_characters:
            existing_entry = merged_characters[character_key]
            existing_entry['aliases'] = list(set(existing_entry['aliases'] + entry['aliases']))
            existing_entry['fragment'].append(entry['fragment'])
        else:
            # Otherwise, add new entry
            merged_characters[character_key] = {
                'character': entry['character'],
                'aliases': entry['aliases'],
                'fragment': [entry['fragment']]
            }

    # Convert merged data to list
    return list(merged_characters.values())

def process_json_file(input_file_path, output_file_path):
    # Read data from the input JSON file
    entries = read_json(input_file_path)
    
    # Merge entries
    merged_data = merge_entries(entries)
    
    # Write merged data to the output JSON file
    write_json(merged_data, output_file_path)


In [ ]:

# Usage
input_file_path = 'data/Merged_Entity_json/Interstellar (film)/Interstellar (film)_Merged.json'
output_file_path = 'data/Deduped_Entity_json/Interstellar/Interstellar_Deduped.json'

process_json_file(input_file_path, output_file_path)


In [ ]:
# Latest deduping usinf llm SEMANTIC DEDUPING working code
from openai import OpenAI
import re
client = OpenAI()


def read_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    # print(data)
    return data

# Function to write to a JSON file
def write_json(file_path, data):
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)
        



def process_json_with_openai(title, json_input):
    # completion = client.chat.completions.create(

    #     messages=[
    #         {
    #             "role": "system",
    #             "content": "You are a helpful assistant skilled in understanding complex character relationships and providing guidance on merging similar characters into single entries. Given a JSON object list of characters, their aliases, and descriptions(fragments), identify duplicates and suggest how their data might be merged. The aim is to reduce duplication and create a cohesive character profile for each unique individual. Output the deduped JSON Object. Make semantic deduping."
    #         },

    #         {
    #             "role": "user",
    #             "content": f"The title of the show is '{title}'. Here is the character data: {json_input}. For now return a json onject with removed duplication, also concatinate there fragments and aliases if found sematically same",
    #         }
    #     ],
    #     model="gpt-3.5-turbo-16k"
    # )
    completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant skilled in understanding complex character relationships and providing guidance on merging similar characters into single entries. Given a JSON object list of characters, their aliases, and descriptions(fragments), identify duplicates and suggest how their data might be merged. The aim is to reduce duplication and create a cohesive character profile for each unique individual. Output the deduped JSON Object. Make semantic deduping."
        },

        {
            "role": "user",
            "content": f"The title of the show is '{title}'. Here is the character data: {json_input}. For now return a json onject with removed duplication, also concatinate there fragments if found similar character",
        }
    ],
    model="gpt-3.5-turbo-16k"
    )
    res=completion.choices[0].message.content
    # print(res)
    return res





In [ ]:

input_file_path = 'data/Merged_Entity_json/Interstellar (film)/Interstellar (film)_Merged.json'  # Path to your input JSON file
output_file_path = 'data/Deduped_Entity_json/Interstellar/Interstellar_Desuped_llm3.json'  # Path to save the output JSON file

# Read the input JSON file
input_data = read_json(input_file_path)
title = "Interstellar"
# Process the JSON with OpenAI
output_data = process_json_with_openai(title,(input_data))
print(output_data)

json_array_match = re.search(r'\[\s*\{[\s\S]*?\}\s*\]', output_data)
if json_array_match:
    json_array_str = json_array_match.group(0)
    print("Extracted JSON Array String:\n", json_array_str)
else:
    print("No JSON array found in the response")
    json_array_str = '[]'  # Fallback to an empty list if no JSON is found

# Parse the extracted JSON array
try:
    parsed_data = json.loads(json_array_str)
    print("Parsed Data:\n", parsed_data)
except json.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")
    parsed_data = []


# Write the output JSON file
write_json(output_file_path, parsed_data)

print(f"Processed JSON has been saved to {output_file_path}")